# RVC Inference noGUI

https://github.com/mamang74/RVC_Inference_noGUI

## 참고한 프로젝트

* [RVC-Studio](https://github.com/SayanoAI/RVC-Studio)
* [Retrieval-based-Voice-Conversion-WebUI](https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI)

# Dependencies
가장 먼저 실행해주세요!

In [ ]:
# @title Clone repositories
!git clone https://github.com/SayanoAI/RVC-Studio
%cd RVC-Studio

In [ ]:
# @title Remove unnecessary files
!rm -rf pages
!rm Home.py RVC_Studio.ipynb conda* docker* Dockerfile requirements.txt

In [ ]:
# @title Clone inference code & models

!wget https://raw.githubusercontent.com/mamang74/RVC_Inference_noGUI/main/inference_cli.py

# @markdown torch 버전을 1.13.1(기존 RVC에서 사용하는 버전) 로 설정합니다. 추론중 오류가 발생한다면 체크하고 코랩을 초기화 해주세요.
USE_OLE_TORCH = False # @param {type:"boolean"}

if USE_OLE_TORCH:
    !wget https://raw.githubusercontent.com/mamang74/RVC_Inference_noGUI/main/requirements_torch.txt
    !mv requirements_torch.txt requirements.txt
else:
    !wget https://raw.githubusercontent.com/mamang74/RVC_Inference_noGUI/main/requirements_nontorch.txt
    !mv requirements_nontorch.txt requirements.txt

!mv requirements_new.txt requirements.txt

!wget https://raw.githubusercontent.com/mamang74/RVC_Inference_noGUI/main/download_models.sh
!chmod 777 download_models.sh
!bash download_models.sh

In [ ]:
# @title Install Dependencies
!pip install -r requirements.txt

In [ ]:
# @title Mount Drive
from google.colab import drive
drive.mount('/content/drive')

import os

if not os.path.isdir("/content/drive/MyDrive/RVC_Inference_noGUI"):
    !mkdir /content/drive/MyDrive/RVC_Inference_noGUI

if not os.path.isdir("/content/drive/MyDrive/RVC_Inference_noGUI/models"):
    !mkdir /content/drive/MyDrive/RVC_Inference_noGUI/models

if not os.path.isdir("/content/drive/MyDrive/RVC_Inference_noGUI/input"):
    !mkdir /content/drive/MyDrive/RVC_Inference_noGUI/input
    

# Model setting & Inference
모델과 음성을 설정 후 추론합니다.


In [ ]:
#@title Copy Model from Drive

# @markdown 드라이브에서 `RVC_Inference_noGUI/models` 폴더 안에 RVC 모델 파일이 담긴 압축파일을 넣어주세요.<br/>압축파일은 다음과 같이 구성되어야 합니다.

# @markdown `Some_model.zip`<br/>`├ Some_model.pth`<br/>`└ added_IVF~~(이름 변경 없이).index`

# @markdown 또는

# @markdown `Some_model.zip`<br/>`└ Some_model`<br/>　　`├ Some_model.pth`<br/>　　`└  added_IVF~~(이름 변경 없이).index`

# @markdown 여기에 압축파일 이름을 적어주세요.
ZIP_NAME = "Some_model.zip" # @param {type:"string"}

# @markdown 불러온 모델을 사용하여 추론을 진행하니 혹시 모델이 바뀐다면 여기서 모델을 다시 설정한 후 실행해주세요.

MODEL_PATH = ""

import os

def CheckFolder():
    global ZIP_NAME, MODEL_PATH

    ZipNameOnly = ZIP_NAME[:ZIP_NAME.rindex(".")]

    if not os.path.isdir("/content/drive/MyDrive/RVC_Inference_noGUI"):
        !mkdir /content/drive/MyDrive/RVC_Inference_noGUI
        !mkdir /content/drive/MyDrive/RVC_Inference_noGUI/models
        print("RVC_Inference_noGUI 폴더가 드라이브에 없어서 새로 만들었습니다. 모델 압축파일을 models 폴더 안에 넣어주세요.")
        return

    if not os.path.isdir("/content/drive/MyDrive/RVC_Inference_noGUI/models"):
        !mkdir /content/drive/MyDrive/RVC_Inference_noGUI/models
        print("RVC_Inference_noGUI/models 폴더가 드라이브에 없어서 새로 만들었습니다. 모델 압축파일을 이 폴더에 넣어주세요.")
        return

    zip_path = "/content/drive/MyDrive/RVC_Inference_noGUI/models/" + ZIP_NAME

    if not os.path.isfile(zip_path):
        print("압축파일을 찾을 수 없습니다! 이름을 다시 확인해주세요.")
        return

    target_path = "/content/RVC-Studio/models/RVC/model.zip"

    !cp $zip_path $target_path
    %cd /content/RVC-Studio/models/RVC
    !unzip model.zip
    !rm $target_path

    if os.path.isdir("/content/RVC-Studio/models/RVC/" + ZipNameOnly):
        p = "/content/RVC-Studio/models/RVC/" + ZipNameOnly
        %cd $p
        !mv *.pth ../
        !mv *.index ../.index/
        %cd ..
        !rm -rf $p

    else:
        !mv *.index .index/

    MODEL_PATH = "/content/RVC-Studio/models/RVC/" + ZipNameOnly + ".pth"


CheckFolder()

print("Use model : " + MODEL_PATH)

In [ ]:
# @title Inference

# @markdown #음성 설정

# @markdown 드라이브에서 `RVC_Inference_noGUI/input` 폴더 안에 추론에 사용할 wav 파일을 넣어주세요.<br/>wav가 아닐 시 변환하여 넣어주세요.

# @markdown 여기에 파일 이름을 적어주세요.
INPUT_NAME = "Some_name.wav" # @param {type:"string"}

INPUT_PATH = ""

def CheckInput():
    global INPUT_NAME

    if not os.path.isdir("/content/drive/MyDrive/RVC_Inference_noGUI"):
        !mkdir /content/drive/MyDrive/RVC_Inference_noGUI
        !mkdir /content/drive/MyDrive/RVC_Inference_noGUI/input
        raise Exception("RVC_Inference_noGUI 폴더가 드라이브에 없어서 새로 만들었습니다. wav 파일을 input 폴더 안에 넣어주세요.")
        return

    if not os.path.isdir("/content/drive/MyDrive/RVC_Inference_noGUI/input"):
        !mkdir /content/drive/MyDrive/RVC_Inference_noGUI/input
        raise Exception("RVC_Inference_noGUI/input 폴더가 드라이브에 없어서 새로 만들었습니다. wav 파일을 이 폴더에 넣어주세요.")
        return

    wav_path = "/content/drive/MyDrive/RVC_Inference_noGUI/input/" + INPUT_NAME

    if not os.path.isfile(wav_path):
        raise Exception("파일을 찾을 수 없습니다! 이름을 다시 확인해주세요.")
        return

    global INPUT_PATH
    INPUT_PATH = wav_path


CheckInput()

# @markdown ---

# @markdown ##RVC 추론 설정
# @markdown 위에서 추가한 모델을 사용하여 추론합니다. <br/> 결과는 자동으로 드라이브에 `RVC_Inference_noGUI/output` 에 업로드됩니다.

# @markdown 각각의 설정들의 설명으로 기존 RVC의 설명들을 써놓았습니다.

# @markdown ---
# @markdown Recommended +7 or +12 key for male to female conversion, and -12 or -5 key for female to male conversion. If the sound range goes too far and the voice is distorted, you can also adjust it to the appropriate range by yourself.
f0_up_key = 0 # @param {type:"slider", min:-12, max:12, step:1}

# @markdown ---
# @markdown Select the pitch extraction algorithm ('crepe': better quality but GPU intensive, 'rmvpe': fast and best quality, and little GPU requirement, others: use them if you really want to...)
f0_method = "rmvpe" # @param ["crepe", "rmvpe", "mangio-crepe", "rmvpe+", "dio", "harvest"]

# @markdown ---
# @markdown Merge Type
merge_type = "median" # @param ["median", "mean"]

# @markdown ---
# @markdown Enable autotuning of extracted pitch
f0_autotune = False # @param {type:"boolean"}

# @markdown ---
# @markdown Resample the output audio in post-processing to the final sample rate. Set to 0 for no resampling:
resample_sr = "0" # @param ["0", "16000", "24000", "22050", "40000", "44100", "48000"]

# @markdown ---
# @markdown Search feature ratio (controls accent strength, too high has artifacting):
index_rate = 0.75 # @param {type:"slider", min:0, max:1, step:0.05}

# @markdown ---
# @markdown If >=3: apply median filtering to the harvested pitch results. The value represents the filter radius and can reduce breathiness.
filter_radius = 3 # @param {type:"slider", min:0, max:7, step:1}

# @markdown ---
# @markdown Adjust the volume envelope scaling. Closer to 0, the more it mimicks the volume of the original vocals. Can help mask noise and make volume sound more natural when set relatively low. Closer to 1 will be more of a consistently loud volume:
rms_mix_rate = 0.2 # @param {type:"slider", min:0, max:1, step:0.05}

# @markdown ---
# @markdown Protect voiceless consonants and breath sounds to prevent artifacts such as tearing in electronic music. Set to 0.5 to disable. Decrease the value to increase protection, but it may reduce indexing accuracy:
protect = 0.2 # @param {type:"slider", min:0, max:0.5, step:0.01}

if not os.path.isdir("/content/drive/MyDrive/RVC_Inference_noGUI/output"):
    !mkdir /content/drive/MyDrive/RVC_Inference_noGUI/output

exec_arg = "python inference_cli.py " + '"' + MODEL_PATH + '" "' + INPUT_PATH + '" ' + str(f0_up_key) + ' "[\\"' + f0_method + '\\"]" ' + str(f0_autotune) + " " + merge_type + " " + str(index_rate) + " " + str(filter_radius) + " " + resample_sr + " " + str(rms_mix_rate) + " " + str(protect) + " " + "/content/drive/MyDrive/RVC_Inference_noGUI/output/"

%cd /content/RVC-Studio
!$exec_arg

print("결과가 드라이브에 저장되었습니다.")